In [1]:
# Cell 1: Imports and Data Loading
import os
import json
import joblib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import gc
from tqdm import tqdm

# TensorFlow / Keras imports
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.regularizers import l1_l2

# Sklearn imports
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import RobustScaler, StandardScaler

import itertools

tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# For reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# Load data (using winter data as in your original code)
data_path = 'data/data_3dcp_wClean.csv'
data = pd.read_csv(data_path)

# Basic information
print("Data shape:", data.shape)
print(data.head())

Data shape: (650, 5)
   layer_height  extrusion  speed  layer_width  over-under
0             8          2     80         49.0           1
1             8          2     80         48.3           1
2             8          2     80         48.4           1
3             8          2     80         48.8           1
4             8          2     80         48.4           1


In [2]:
# Cell 2: Data Preprocessing - Train/Test Split
# Define features and target
X = data[['extrusion', 'layer_height', 'layer_width']]
y = data['speed']

# Split data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print("Training set shape:", X_train.shape)
print("Test set shape:", X_test.shape)

Training set shape: (520, 3)
Test set shape: (130, 3)


In [3]:
# Cell 3: Define the Neural Network Model Function
def create_nn_model(input_dim, model_depth=2, layer_size=32, l1_rate=1e-3, l2_rate=1e-3,
                    dropout_rate=0.0, learning_rate=1e-2):
    model = Sequential()
    model.add(Input(shape=(input_dim,)))
    
    for _ in range(model_depth):
        model.add(Dense(layer_size, activation='elu',
                        kernel_regularizer=l1_l2(l1_rate, l2_rate)))
        model.add(Dropout(dropout_rate))
    
    model.add(Dense(1))
    model.compile(optimizer=Adam(learning_rate=learning_rate), loss='mean_squared_error')
    return model

In [4]:
# Cell 4: Hyperparameter Grid Search with Caching, Detailed Logging, and Memory Clearance

import os
import json
import itertools
import gc
from tqdm import tqdm

# Define parameter grid (including batch_size)
param_grid = {
    'model_depth': [2, 3],
    'layer_size': [32, 64, 96],
    'dropout_rate': [0.0, 0.2],
    'batch_size': [64, 96, 128],
    'learning_rate': [1e-2, 1e-3],
    'l1_rate': [0.0, 1e-3],
    'l2_rate': [0.0, 1e-3]
}

# Define the scalers to try
scaler_options = {
    'robust': RobustScaler(),
    'standard': StandardScaler()
}

# Ensure SAVE_DIR exists
SAVE_DIR = 'model'
os.makedirs(SAVE_DIR, exist_ok=True)
results_json_path = os.path.join(SAVE_DIR, "hyperparameter_search_results.json")

# Load existing results if available; otherwise, initialize empty list.
if os.path.exists(results_json_path):
    with open(results_json_path, 'r') as f:
        results_all = json.load(f)
else:
    results_all = []

# Create a set of keys for already tested combinations.
# The key is a tuple: (scaler, model_depth, layer_size, dropout_rate, batch_size, learning_rate, l1_rate, l2_rate)
tested_keys = set()
for res in results_all:
    key = (res['scaler'], res['model_depth'], res['layer_size'], res['dropout_rate'],
           res['batch_size'], res['learning_rate'], res['l1_rate'], res['l2_rate'])
    tested_keys.add(key)

# Callbacks for training
early_stopping = EarlyStopping(monitor='val_loss', patience=30, restore_best_weights=True, verbose=0)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2, patience=10, min_lr=1e-5, verbose=0)

# Calculate total iterations for the tqdm progress bar
total_iterations = len(scaler_options) * (
    len(param_grid['model_depth']) *
    len(param_grid['layer_size']) *
    len(param_grid['dropout_rate']) *
    len(param_grid['batch_size']) *
    len(param_grid['learning_rate']) *
    len(param_grid['l1_rate']) *
    len(param_grid['l2_rate'])
)
pbar = tqdm(total=total_iterations, desc="Hyperparameter search progress", unit="iter")

iteration_count = 0

# Iterate over scaler options and hyperparameter combinations
for scaler_name, scaler_instance in scaler_options.items():
    # Fit the scaler on the training data and transform train data
    X_train_scaled = scaler_instance.fit_transform(X_train)
    
    # Loop over all hyperparameter combinations
    for depth, layer_size, dropout_rate, batch_size, learning_rate, l1_rate, l2_rate in itertools.product(
            param_grid['model_depth'],
            param_grid['layer_size'],
            param_grid['dropout_rate'],
            param_grid['batch_size'],
            param_grid['learning_rate'],
            param_grid['l1_rate'],
            param_grid['l2_rate']):
        
        iteration_count += 1
        # Create a unique key for the current parameter combination
        current_key = (scaler_name, depth, layer_size, dropout_rate, batch_size, learning_rate, l1_rate, l2_rate)
        
        # Check if this combination has already been tested
        if current_key in tested_keys:
            print(f"Iteration {iteration_count}: Combination {current_key} already tested. Skipping.")
            pbar.update(1)
            continue
        
        # Define parameters dictionary for this iteration
        params = {
            'iteration': iteration_count,
            'scaler': scaler_name,
            'model_depth': depth,
            'layer_size': layer_size,
            'dropout_rate': dropout_rate,
            'batch_size': batch_size,
            'learning_rate': learning_rate,
            'l1_rate': l1_rate,
            'l2_rate': l2_rate
        }
        print(f"Iteration {iteration_count}: Training with params: {params}")
        
        # Create and train the model
        model = create_nn_model(input_dim=X_train.shape[1],
                                model_depth=depth,
                                layer_size=layer_size,
                                dropout_rate=dropout_rate,
                                learning_rate=learning_rate,
                                l1_rate=l1_rate,
                                l2_rate=l2_rate)
        
        history = model.fit(X_train_scaled, y_train, epochs=1000, batch_size=batch_size,
                            verbose=0, validation_split=0.2, callbacks=[reduce_lr, early_stopping])
        
        best_val_loss = min(history.history['val_loss'])
        epochs_trained = len(history.history['loss'])
        params.update({'val_loss': best_val_loss, 'epochs_trained': epochs_trained})
        
        print(f"Iteration {iteration_count}: Completed with best_val_loss: {best_val_loss:.5f} after {epochs_trained} epochs\n")
        
        # Append the new result and update the tested keys
        results_all.append(params)
        tested_keys.add(current_key)
        
        # Save the updated results to the JSON file after each new test
        with open(results_json_path, 'w') as f:
            json.dump(results_all, f, indent=4)
        
        pbar.update(1)
        
        # Clear session and garbage collect to free memory before the next iteration
        tf.keras.backend.clear_session()
        gc.collect()

pbar.close()

# Convert all results to a DataFrame and print top 5 combinations
results_df = pd.DataFrame(results_all)
print("\nTop 5 hyperparameter combinations:")
print(results_df.sort_values('val_loss').head())

Hyperparameter search progress:   0%|          | 0/576 [00:00<?, ?iter/s]

Iteration 1: Combination ('robust', 2, 32, 0.0, 64, 0.01, 0.0, 0.0) already tested. Skipping.
Iteration 2: Combination ('robust', 2, 32, 0.0, 64, 0.01, 0.0, 0.001) already tested. Skipping.
Iteration 3: Combination ('robust', 2, 32, 0.0, 64, 0.01, 0.001, 0.0) already tested. Skipping.
Iteration 4: Combination ('robust', 2, 32, 0.0, 64, 0.01, 0.001, 0.001) already tested. Skipping.
Iteration 5: Combination ('robust', 2, 32, 0.0, 64, 0.001, 0.0, 0.0) already tested. Skipping.
Iteration 6: Combination ('robust', 2, 32, 0.0, 64, 0.001, 0.0, 0.001) already tested. Skipping.
Iteration 7: Combination ('robust', 2, 32, 0.0, 64, 0.001, 0.001, 0.0) already tested. Skipping.
Iteration 8: Combination ('robust', 2, 32, 0.0, 64, 0.001, 0.001, 0.001) already tested. Skipping.
Iteration 9: Combination ('robust', 2, 32, 0.0, 96, 0.01, 0.0, 0.0) already tested. Skipping.
Iteration 10: Combination ('robust', 2, 32, 0.0, 96, 0.01, 0.0, 0.001) already tested. Skipping.
Iteration 11: Combination ('robust', 

KeyboardInterrupt: 

In [ ]:
# Cell 5: Visualization of Hyperparameter Search Results
sns.set_theme(style="whitegrid", context="notebook")

# Visualize validation loss vs. layer size, differentiated by scaler and model depth
plt.figure(figsize=(10, 6))
sns.scatterplot(data=results_df, x='layer_size', y='val_loss', hue='scaler', style='model_depth', s=100)
plt.title("Validation Loss vs. Layer Size (by scaler and model depth)")
plt.xlabel("Layer Size")
plt.ylabel("Validation MSE")
plt.legend(title="Scaler / Model Depth")
plt.show()

# Pairplot to check relationships among hyperparameters and performance
sns.pairplot(results_df, vars=['scaler', 'model_depth', 'layer_size', 'dropout_rate', 'learning_rate', 'l1_rate', 'l2_rate', 'val_loss'])
plt.suptitle("Pairplot of Hyperparameters and Validation Loss", y=1.02)
plt.show()

In [ ]:
# Cell 6: Training and Evaluating the Best Model on the Test Set
# Identify the best hyperparameter combination (lowest validation loss)
best_params = results_df.sort_values('val_loss').iloc[0].to_dict()
print("Best hyperparameters found:")
print(best_params)

# Retrieve the best scaler option and fit it on the entire training data
if best_params['scaler'] == 'robust':
    best_scaler = RobustScaler()
elif best_params['scaler'] == 'standard':
    best_scaler = StandardScaler()
else:
    raise ValueError("Scaler type not recognized.")

X_train_scaled_best = best_scaler.fit_transform(X_train)
X_test_scaled_best = best_scaler.transform(X_test)

# Recreate and train the best model using the best scaler
best_model = create_nn_model(input_dim=X_train.shape[1],
                             model_depth=int(best_params['model_depth']),
                             layer_size=int(best_params['layer_size']),
                             dropout_rate=best_params['dropout_rate'],
                             learning_rate=best_params['learning_rate'],
                             l1_rate=best_params['l1_rate'],
                             l2_rate=best_params['l2_rate'])

history_best = best_model.fit(X_train_scaled_best, y_train, epochs=1000, batch_size=64,
                              verbose=0, validation_split=0.2, callbacks=[reduce_lr, early_stopping])

# Plot training history
plt.figure(figsize=(8, 5))
plt.plot(history_best.history['loss'], label='Train Loss')
plt.plot(history_best.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Mean Squared Error')
plt.title('Training History of the Best Model')
plt.legend()
plt.show()

# Evaluate on the test set
test_loss = best_model.evaluate(X_test_scaled_best, y_test, verbose=0)
print("Test set Mean Squared Error:", test_loss)

In [ ]:
# Cell 7: Save Best Parameters, Model, Scaler, and All Hyperparameter Search Results
SAVE_DIR = 'model'
os.makedirs(SAVE_DIR, exist_ok=True)

# Save the best hyperparameters to a JSON file
best_params_path = os.path.join(SAVE_DIR, "best_hyperparameters.json")
with open(best_params_path, 'w') as f:
    json.dump(best_params, f, indent=4)
print("Best hyperparameters saved to:", best_params_path)

# Save the best model
model_filename = '3DCP_wANN_best.h5'
best_model.save(os.path.join(SAVE_DIR, model_filename))
print("Best model saved to:", os.path.join(SAVE_DIR, model_filename))

# Save the scaler using joblib
scaler_filename = f"scaler_{best_params['scaler']}.pkl"
joblib.dump(best_scaler, os.path.join(SAVE_DIR, scaler_filename))
print("Scaler saved to:", os.path.join(SAVE_DIR, scaler_filename))

# Save the entire hyperparameter search results as CSV
results_csv_path = os.path.join(SAVE_DIR, "hyperparameter_search_results.csv")
results_df.to_csv(results_csv_path, index=False)
print("Hyperparameter search results saved to CSV at:", results_csv_path)

# Save the entire hyperparameter search results as JSON for additional documentation
results_json_path = os.path.join(SAVE_DIR, "hyperparameter_search_results.json")
results_df.to_json(results_json_path, orient="records", indent=4)
print("Hyperparameter search results saved to JSON at:", results_json_path)